### CS5228 Assignment 3

Hello everyone, this assignment notebook covers Graph Mining. There are some code-completion tasks and question-answering tasks in this answer sheet. For code completion tasks, please write down your answer (i.e., your lines of code) between sentences that "Your code starts here" and "Your code ends here". The space between these two lines does not reflect the required or expected lines of code. For answers in plain text, you can refer to [this Markdown guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd) to customize the layout (although it shouldn't be needed).

When you work on this notebook, you can insert additional code cells (e.g., for testing) or markdown cells (e.g., to keep track of your thoughts). However, before the submission, please remove all those additional cells again. Thanks!

**Important:** 
* Remember to rename and save this Jupyter notebook as **A3_YourName_YourNUSNETID.ipynb** (e.g., **A3_BobSmith_e12345678.ipynb**) before submission! Failure to do so will yield a penalty of 1 Point.
* Remember to rename and save the script file *A3_script.py* as **A3_YourName_YourNUSNETID.py** (e.g., **A3_BobSmith_e12345678.py**) before submission! Failure to do so will yield a penalty of 1 Point.
* Submission deadline is Oct 30, 11.59 pm. Late submissions will be penalized by 10% for each additional day.

Please also add your nusnet and student id in the code cell below. This is just to make any identification of your notebook doubly sure.

In [30]:
student_id = 'A0248326B'
nusnet_id = 'E0925478'

Here is an overview over the tasks to be solved and the points associated with each task. The notebook can appear very long and verbose, but note that a lot of parts provide additional explanations, documentation, or some discussion. The code and markdown cells you are supposed to complete are well, but you can use the overview below to double-check that you covered everything.

* **1 Recommender Systems (20 Points)**
    * 1.1 Matrix Factorization (15 Points)
        * 1.1 a) Implement method `fit()` (8 Points)
        * 1.1 b) Explore different hyperparameter settings (2 Points)        
        * 1.1 c) Matrix Factorization & Updates (5 Points)
    * 1.2 Questions about Recommender Systems (5 Points)
* **2 Graph Mining (27 Points)**
    * 2.1 Centrality (20 Points)
        * 2.1 a) Implement Closeness Centrality (4 Points)
        * 2.1 b) Implement PageRank Centrality (7 Points
        * 2.1 c) Comparing Centrality Measures (5 Points)
        * 2.1 d) Discussion of Limitation of Results (4 Points)
    * 2.2 Community Detection (10 Points)
        * 2.2 a) Implement Girvan-Newman Algorithm (4 Points)
        * 2.2 b) Question about Girvan-Newman Algorithm (6 Points)

## Setting up the Notebook

Using this mode of visualization will allow you zoom in into plot, which will be convenient for some tasks.

In [31]:
%matplotlib notebook

In [32]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
import pandas as pd
import networkx as nx

from src.utils import plot_mrt_graph


**Important:** In this notebook, most code-completion tasks require to edit the file `A3_script.py`. The code cell below ensures that any change to the file (after saving) will cause a reload in this notebook. So there's no need to "manually" import the code after every change. Way more convenient.

In [34]:
from A3_GanKaiyuan_E0925478 import closeness, pagerank, girvan_newman, NMF
#from A3_BobSmith_e12345678 import closeness, pagerank, girvan_newman, NMF

## 1 Recommender Systems

### 1.1 Matrix Factorization (15 Points)

Matrix Factorization -- and here more specifically: non-negative Matrix Factorization -- is a class of algorithms where a matrix $M$ is factorized into (usually) two matrices $W$ and $H$, with the property that all three matrices have no negative elements. Matrix Factorization is popular techniques applied in recommender systems, where $W$ and $H$ contain a latent representation of all users and all items, respectively, and $M$ represents the rating matrix.

In this task, you will implement (non-negative) Matrix Factorization from scratch using Gradient Descent as covered in the lecture. In fact, we use the rating matrix $M$ which was used as an example in the lecture:

In [35]:
M = np.array([
    [4, 0, 0, 5, 1, 0, 0],
    [5, 5, 4, 0, 0, 0, 0],
    [0, 0, 0, 2, 4, 5, 0],
    [0, 3, 0, 0, 0, 0, 3]
], dtype=np.float64)

print(M)

[[4. 0. 0. 5. 1. 0. 0.]
 [5. 5. 4. 0. 0. 0. 0.]
 [0. 0. 0. 2. 4. 5. 0.]
 [0. 3. 0. 0. 0. 0. 3.]]


We provide you with the skeleton code for class `NMF` (short for Non-Negative Matrix Factorization). The code includes the initialization of matrices `W` and `H`, as well as of Matrix `Z`. Matrix `Z` is an auxiliary matrix containing the indices of all non-zero entries of Matrix `M`. Recall from the lecture that we need to compute the Gradient Descent only based on the non-zero entries in the rating matrix.

The code cell below shows an example using the default parameter (`k=100`). The shapes of `W` and `H` reflect the number of users and items, as well as the size $k$ of the latent representations. The shape of `Z` is `(num_nonzero, 2)`. For example matrix `M`, the shape should be `(11, 2)` since `M` has 11 non-zero entries.


In [36]:
np.random.seed(0)

nmf = NMF(M)

print('W.shape = {}'.format(nmf.W.shape))
print('H.shape = {}'.format(nmf.H.shape))
print('Z.shape = {}'.format(nmf.Z.shape))
print()
print('Z containing all the indices of all non-zero entries in M (first 5 entries only)')
print(nmf.Z[:5])

W.shape = (4, 100)
H.shape = (100, 7)
Z.shape = (11, 2)

Z containing all the indices of all non-zero entries in M (first 5 entries only)
[[0 0]
 [0 3]
 [0 4]
 [1 0]
 [1 1]]


We also provide you with the method `calc_loss()` which calculates the loss w.r.t. the current values of matrices `W` and `H`. **Important:** Note that method implements the loss without regularization! Since we need this method only to print the loss and so to see its trend over time, this simplified calculation is sufficient.

In [37]:
np.random.seed(0)

nmf = NMF(M)

loss = nmf.calc_loss()

print('Initial loss: {:.1f}'.format(loss))

Initial loss: 4879.6


You should see an initial loss of **4879.6**.

**1.1 a) Implement method `fit()` to perform matrix factorization using Gradient Descent! (8 Points).** The complete algorithm together with the required gradients is available as pseudo code in the lecture slides, and you are already familiar with the basic concept of Gradient Descent. Here, consider the regularization terms when calculating the gradients.

In [38]:
np.random.seed(0)

nmf = NMF(M)

nmf.fit(verbose=True)

Loss: 4718.82797 	 0%
Loss: 3509.27814 	 10%
Loss: 2761.96374 	 20%
Loss: 2268.78939 	 30%
Loss: 1927.21239 	 40%
Loss: 1681.70977 	 50%
Loss: 1499.99964 	 60%
Loss: 1362.24042 	 70%
Loss: 1255.68257 	 80%
Loss: 1171.83133 	 90%
Loss: 1110.92019 	 100%


With the default values for all parameters  (`k=100`, `learning_rate=0.0001`, `lambda_reg=0.1`, `num_iter=100`), you should see a loss around **167.6** at the end of the training.

**Predicting unknown ratings (nothing for you to do here).** With our learned estimates for `W` and `H`, we can simply calculate matrix `P` as the product of `W` and `H`, representing the matrix of predicted ratings. We encapsulate this simple computation in method `predict()`.

In [39]:
P = nmf.predict()

print(np.around(P, 2))

[[ 8.25 11.53 14.5   8.38 14.61 15.5  17.86]
 [ 7.59  6.43  8.19  9.96 13.08 17.17 16.18]
 [14.3  12.55 16.    9.29 18.77 20.62 19.76]
 [12.24  9.28 15.11 13.54 16.9  19.56 20.76]]


With the default values for all parameters  (`k=100`, `learning_rate=0.0001`, `lambda_reg=0.1`, `num_iter=100`), the result should look something like this:

```
[[ 7.02 10.17 11.97  7.85  5.61 10.61 12.52]
 [ 7.75  6.9   8.05 11.22  9.09 14.9  13.09]
 [ 9.65  8.96 10.37  7.02  6.81  8.33 10.76]
 [ 9.11  7.25 10.69 11.67  9.07 12.4   9.27]]
```

**1.2 d) Explore different hyperparameter settings and briefly explain your observations! (2 Points)** You can use the code cell below for that; you can simply set different values for `k`, `learning_rate`, `lambda_reg`, and `num_iter`. Note that it's not about find the *best* values for those parameters but to observe how changing those values affect the result.

**Your Answer:**
- By increase learning rate, the model reaches to a lower loss with a fast pace.
- By increasing the num_iters, the model reaches to a lower loss but with a slow pace.
- With lower lambda_reg, the model reaches to a lower loss.

In [40]:
np.random.seed(0)

k, learning_rate, lambda_reg, num_iter = 100, 0.01, 0.001, 100

nmf = NMF(M, k=k)

nmf.fit(learning_rate=learning_rate, lambda_reg=lambda_reg, num_iter=num_iter, verbose=True)

P = nmf.predict()

print('\nReconstructed rating matrix:')
print(np.around(P, 2))

Loss: 735.52455 	 0%
Loss: 606.93092 	 10%
Loss: 606.79611 	 20%
Loss: 606.66399 	 30%
Loss: 606.53141 	 40%
Loss: 606.39886 	 50%
Loss: 606.26634 	 60%
Loss: 606.13385 	 70%
Loss: 606.00139 	 80%
Loss: 605.86897 	 90%
Loss: 605.74981 	 100%

Reconstructed rating matrix:
[[ 4.    6.48 10.82  5.   10.84 11.29 13.44]
 [ 5.    5.    4.    6.17 10.54 14.07 13.05]
 [ 8.24  7.18 12.17  2.   14.84 16.01 15.39]
 [ 7.91  3.   13.45  9.94 15.63 18.4  19.43]]


#### 1.1 e) Matrix Factorization & Updates (5 Points)

You have now implemented a basic model-based recommender system using (non-negative) Matrix Factorization. Since we used only a toy rating matrix, performance was not an issue here. In real-world recommendations with many users and items, Matrix Factorization can be quite time consuming. The problem is that online platforms are very dynamic: users are joining and leaving, new items are added, users add new or update previous ratings. All of those cases change the rating matrix.

**How do different cases (e.g., new user/item/rating) affect a current result of a Matrix Factorization for a recommender system? (3 Points)** Outline the different problems, and discuss meaningful approaches to mitigate them. For example, a new user or item refers to the *Cold-Start Problem*. What are good practical strategies to address the Cold-Start Problem and other changes to the rating matrix using Matrix Factorization?

(Note: When you're discussing challenges regarding runtime/performance, please **exclude** any solutions relying on bigger clusters and parallel computing :). While those are valid points, in principle, here we want to focus on conceptual solutions).

**Your Answer:**

### 1.2 Questions about Recommender Systems (5 Points)

**True/False Questions about Recommender Systems**: In the table below are 5 statements that are either True or False. Complete the table to specify whether a statement is True or False, and provide a brief explanation for your answer (Your explanation is more important than a simple True/False answer)

**Your Answer:**

This is a markdown cell. Please fill in your answers for (1)~(5).

| No. | Statement                                                                                                   | True or False? | Brief Explanation                                                                                               |
|-----|------------------------------------------------------------------------------------------------------------|----------------|-----------------------------------------------------------------------------------------------------------------|
| (1)  | You have a running recommender system using 1-5 star ratings. Now you change it to a 1-10 scoring system. This makes your 1-5 star ratings obsolete. | False          | 1-5 stars can be mapped to 1-10 scoring system.                                                                 |
| (2)  | Recommendation engines can limit users' exposure to a wider variety of items. | True           | The user can be only recommended with favourable items.                                                         |
| (3)  | Most recommendation engines benefit from some degree of randomization when providing recommendations to a user. | True           | Randomization provide users with more unseen items.                                                             |
| (4)  | A CF-based recommender system will always outperform a Content-Based Recommender System | False          | The performance can be the same under some situation.                                                           |
| (5)  | For User-Based CF, we normalize the ratings to ensure that all ratings of a user sum up to 1. | False     | We normalize the ratings to ensure that bad ratings are represented by negtive value which is more informative. |

------------------

## 2 Graph Mining

### Load and Prepare Data

Throughout this section we work the MRT train network as our underlying graph. The MRT stations mark the nodes, and there is an edge (directed or undirected; see below) if there is a direct train connection between the respective MRT stations.

**Load data from files.** We first load the information about the MRT stations. We only need this information to have access to the latitude and longitude of the stations, so we can plot the MRT graph and preserve the relative geographic locations of the MRT stations.

In [41]:
df_mrt_stations = pd.read_csv('data/a3-mrt-stations.csv')

df_mrt_stations.head()

,code,line,name,opening_year,subzone,latitude,longitude
0,cc1,cc,dhoby ghaut,2010,dhoby ghaut,1.298912,103.846293
1,cc2,cc,bras basah,2010,bras basah,1.296862,103.850667
2,cc3,cc,esplanade,2010,city hall,1.293658,103.855081
3,cc4,cc,promenade,2010,marina centre,1.293998,103.860350
4,cc5,cc,nicoll highway,2010,nicoll,1.299767,103.863637


The following file contains the main information: Which MRT stations are directly connected with by a train. Not that the file contains each connection twice for both directions.

In [42]:
df_mrt = pd.read_csv('data/a3-mrt-connections.csv')

df_mrt.head()

,from,to
0,tanjong pagar,raffles place
1,mayflower,bright hill
2,macpherson,mattar
3,rochor,little india
4,kent ridge,one-north


### Create Graphs

From this data, we can easily create 2 NetworkX graphs. We create an undirected graph `G_undirected` and a direct graph `G_directed`.

In [43]:
## Create an "empty" undirected and directed graph
G_undirected = nx.Graph()
G_directed = nx.DiGraph()

for idx, row in df_mrt.iterrows():
    G_undirected.add_edge(row['to'], row['from'])
    G_directed.add_edge(row['to'], row['from'])

We provide you with the method `plot_mrt_graph()` to visualize the train network. As mentioned before, we can utilize the information about the geocoordinates of MRT station to preserve their relative location. Of course mthe connections between the nodes / MRT stations are still just straight lines.

In [44]:
plot_mrt_graph(G_undirected, df_mrt_stations)

<IPython.core.display.Javascript object>

### 2.1 Centrality (12 Points)

We first explore the concept of Centrality which aims to identify "important" nodes in a Graph. As we saw in the lecture, there is a wider variety of Centrality measures to look at different aspects of a node and the whole graph to compute a node's importance. In the following, we look at *Closeness* and *PageRank*.

#### 2.1 a) Implement Closeness Centrality (4 Points)

The Closeness Centrality of a node $v$ is defined as

$$
closeness(v) = \frac{N}{\sum_{w\in V}d(v,w)}
$$

where $N$ is the number of nodes that can be reached from $v$, and $d(v,w)$ is the length of the shortest path between node $v$ and a node $w$.

We saw that both distance-based centrality measure Closeness and Betweenness require the to solve the All-Pairs Shortest Paths (APSP) problem. Since this is not a "programming" or "algorithms and data structures" module, we don't expect you to come up with your own solution for the problem from scratch. For this task, you can utilize any method from [`nx.algorithms.shortest_paths`](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html). Using a method to compute the shortest path between two nodes will make the computation of Closeness Centrality pretty straightforward.

**Implement method `closeness()` to compute the Closeness Centrality of a Graph G.** You can assume the input Graph G being strongly connected, undirected, and unweighted.

You can use the code cell below to test your implementation.

In [45]:
my_closeness_scores = closeness(G_undirected)

for station, score in sorted(my_closeness_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

little india (0.14064)
botanic gardens (0.14018)
newton (0.13925)
caldecott (0.13730)
stevens (0.13685)


**Compare your implementation with the one from NetworkX**. The code cell belows computes the Closeness Centrality over the *undirected* MRT graph using the implementation from NetworkX, and again shows the 5 MRT stations with the highest scores. Apart from minor precision issues, the NetworkX result and your result should match, of course.

In [46]:
nx_closeness_scores = nx.algorithms.centrality.closeness_centrality(G_undirected)

for station, score in sorted(nx_closeness_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

little india (0.13953)
botanic gardens (0.13907)
newton (0.13816)
caldecott (0.13622)
stevens (0.13578)


#### 2.1 a) Implement PageRank Centrality (8 Points)

In this task, you will implement the basic PageRank algorithm using the Power Iteration methods as introduced in the lecture.

$$
c_{PR} = \alpha M c_{PR} + (1-\alpha)E
$$

where $E = (1/n, 1/n, ..., 1/n)^T$ with $n$ being the number of nodes.

Recall from the lecture that PageRank requires the **transition matrix** of a graph is input. For this, we provide you with the method `create_transition_matrix(A)` that converts the adjacency matrix of a Graph G into an transition matrix. Check out also the given code in method `pagerank()` where we use a numpy method to convert the Graph G to its adjacency matrix and then call `create_transition_matrix(A)`.

**Implement method `pagerank()` to compute the PageRank Centrality of a Graph G**.  You can assume the input Graph G being strongly connected, directed, and unweighted.

You can use the code cell below to test your implementation.

In [47]:
my_pagerank_scores = pagerank(G_directed)

for station, score in sorted(my_pagerank_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

woodlands (0.01423)
dhoby ghaut (0.01393)
tampines (0.01384)
buona vista (0.01303)
serangoon (0.01290)


**Compare your implementation with the one from NetworkX**. The code cell belows computes the PageRank Centrality over the *directed* MRT graph using the implementation from NetworkX, and again shows the 5 MRT stations with the highest scores. Apart from minor precision issues, the NetworkX result and your result should match, of course. Note that your implementation and the one of NetworkX are using the same default value for `alpha` and `eps` (called `tol` in case of NetworkX)

In [48]:
nx_pagerank_scores = nx.pagerank(G_directed)

for station, score in sorted(nx_pagerank_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

woodlands (0.01422)
dhoby ghaut (0.01393)
tampines (0.01384)
buona vista (0.01303)
serangoon (0.01290)


#### 2.1 c) Comparing Centrality Measures (5 Points)

In the lecture, we covered several more Centrality measures. The table below shows the top-5 MRT stations with respect to their scores w.r.t to most of the measures we talked about. Note that we don't care about the exact scores, but just about the ranking these scores induce.


| Rank | PageRank | InDegree | OutDegree | Closeness | Betweenness |
| ---  | ---      | ---      | ---       | ---         |  --- |
| 1    |  woodlands | dhoby ghaut  | dhoby ghaut |little india |  botanic gardens  |
| 2    |  dhoby ghaut  | macpherson | macpherson | botanic gardens | buona vista |
| 3    | tampines  | little india  | little india | newton  | bishan  |
| 4    | buona vista  | buona vista | buona vista | caldecott | serangoon |
| 5    | serangoon | chinatown| chinatown | stevens | caldecott |

**Discuss the rankings and any interesting observations.** Based on the definitions and intuitions behind these 5 different centrality measures, discuss the rankings from the table above: For each centrality measure, briefly describe in your own words what it means for a MRT station to have the highest score!

(Note: Difference between the measures w.r.t. complexity and performance are not relevant here)

**Your Answer:**
- In undirected graph, the inDegree rank is the same as OutDegree Rank. The higher the rank is, the more lines passing by here.
- The station with higher PageRank indicates that the station is passing by some important station.
- The station with higher closeness indicates that the station is closed to all other MRT stations.
- The station with higher betweenness indicates that the station is on most of the shortest path between any two stations. Therefore, such station is very important as it is the connection of many stations.

#### 2.1 d) Discussion of Limitation of Results (4 Points)

The table in 2.1 c) shows us the top-5 MRT stations with respect to different centrality measures. While this is interesting in itself, we usually want this information to make any informed decision depending on a given application scenario. For example, the Government, the Land Transport Authority (LTA), Urban Redevelopment Authority (URA), or private home buyers have very different information needs from the MRT graph.

Come up with **TWO (2)** concrete application scenarios utilizing the MRT graph. For each scenario

* briefly motivate which centrality measure is arguably the most suitable, and

* briefly discuss the practical limitation that might yield subpar results

**Your Answer:**
- For private house buyers, they might want to utilize the betweenness information because living near high-betweenness station makes their daily travel more convenient.
- For the Land Transport Authority, they might utilize the closeness informatino because if their product stores in such station they can transport to other places more easily. Because higher closeness station indicates that the station might locate at the center of Singapore.

### 2.2 Community Detection (10 Points)

#### 2.2 a) Implement Girvan-Newman Algorithm (4 Points)

The Girvan-Newman Algorithm finds communities in a graph by assuming a strongly connected graph and then iteratively removing a minimum set of edges until the graph breaks into 2 components. The criteria to remove an edge is based on the Edge Betweenness Centrality; cf. lecture slides. The Edge Betweenness Centrality $c_{B}(e)$ of an Edge $e$ given a Graph $G=(V,E)$ is defined as:

$$c_{B}(e) = \sum_{u,w\in V} \frac{\sigma(v,w|e)}{\sigma(v,w)}$$

where $\sigma(v,w)$ is the number of shortest paths from $v$ to $w$, and $\sigma(v,w|e)$ is the number of shortest paths from $v$ to $w$ going through Edge $e$.

Similar to the more traditional Betweenness Centrality for nodes, Edge Betweenness Centrality also fundamentally requires solving the All-Pairs Shortest Paths (APSP) problem. As such, we could again utilize [`nx.algorithms.shortest_paths`](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html). However, it gets a bit tedious since we here need **all** shortest paths between **all** pairs of nodes. So you can simply use [`nx.algorithms.centrality.edge_betweenness_centrality`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html) here.

**Implement the method `girvan_newman()` split a Graph G into 2 components!** You can assume that the Graph is undirected, unweighted, and strongly connected. Together with being able to use So you can simply use [`nx.algorithms.centrality.edge_betweenness_centrality`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html), this should make it a rather straightforward task. Here are 2 additional constraints:

* If 2 or more edges have the same maximum edge centrality, still remove only **1 edge** per iteration (you can randomly pick one of the edges with maximum edge centrality)
* Include a print statement that shows which edge has been removed in an iteration (see given code of method `girvan_newman()`); this is merely so you can check your implementation with the expected outcome below.


In [49]:
communities, G_split = girvan_newman(G_undirected, verbose=True)

Edge ('farrer road', 'botanic gardens') removed (edge betweenness centrality: 0.211)
Edge ('outram park', 'tiong bahru') removed (edge betweenness centrality: 0.217)
Edge ('harbourfront', 'outram park') removed (edge betweenness centrality: 0.245)
Edge ('marsiling', 'woodlands') removed (edge betweenness centrality: 0.380)


The expect output for the code cell above is:

```
Edge ('farrer road', 'botanic gardens') removed (edge betweenness centrality: 0.211)
Edge ('outram park', 'tiong bahru') removed (edge betweenness centrality: 0.217)
Edge ('harbourfront', 'outram park') removed (edge betweenness centrality: 0.245)
Edge ('marsiling', 'woodlands') removed (edge betweenness centrality: 0.380)
```

The return Graph `G_split` is the original graph without the edges that needed to be removed to break of the original Graph `G` in to 2 components. As such, we can now plot `G_split` to visualize the 2 components. You need to zoom in to the MRT connections that reflect the removed edges.

In [50]:
plot_mrt_graph(G_split, df_mrt_stations)

<IPython.core.display.Javascript object>

#### 2.2 b) Question to Girvan-Newman Algorithm (6 Points)

This is a markdown cell. Please fill in your answers for (1)~(3), 2 Points each.

| No. | Question                                                                                                   | You Answer                                                                                                                                                                                                                                                                     |
|---------------------------------|--------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| (1)  | `girvan_newman()` removes the edge with the largest Edge Betweenness Centrality value. For the 4 edges that get removed from the MRT Graph, why do the values increase after each iteration? | When removing the largest Edge Betweenness Centrality value, the topology of the graph changes, so that there will be a edge is contained by more shortest paths in the graph.                                                                                                 |
| (2)  | `girvan_newman()` always removes only one edge in each iteration even if 2 or more dges have the same maximum Edge Betweenness Centrality. Why is this meaningful, or why don't we remove all edge with the same maximum Edge Centrality in the same iteration? | We don't remove all edge with same max Edge Centrality because removing one of the edge causes change to the graph so that it might not be the case that the previous max-value edge still has the max value after change.                                                     |
| (3)  | `girvan_newman()` resolves ties by randomly picking one of the edges with the maximum Edge Betweenness Centrality. Are there better/smarter ways to pick which edge to choose |                                                                                                                                                                                                                                                                                |
